In [1]:
using Revise

using RigidBodyDynamics
using RigidBodyDynamics: Bounds

using Plots
using BilevelTrajOpt

In [2]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)
body = findbody(mechanism, "ball")
basejoint = joint_to_parent(body, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100)
μ = 0.5
motion_type = :xyz
point = Point3D(default_frame(findbody(mechanism, "floor")), [0.,0.,0.])
normal = FreeVector3D(default_frame(findbody(mechanism, "floor")), [0.,0.,1.])
hs = HalfSpace(point, normal)
floor = Obstacle(hs, μ, motion_type)
obstacles = [floor]
env = parse_contacts(mechanism, urdf, obstacles)
x0 = MechanismState(mechanism)
Δt = 0.005
sim_data = get_sim_data(x0,env,Δt,true);

In [41]:
q0 = [1., 0., 0., 0., 0., 0., 0.]
v0 = [0., 0., 0., 0.5, 0., 0.]

u0 = zeros(sim_data.num_v)
set_configuration!(x0, q0)
set_velocity!(x0, v0)
setdirty!(x0)
ctrl! = (u,t,x) -> u[:] = 0.
traj = BilevelTrajOpt.simulate(x0,env,sim_data.Δt,1,ctrl!,implicit_contact=false)
qnext = traj[1:sim_data.num_q,2]
vnext = traj[sim_data.num_q+1:sim_data.num_q+sim_data.num_v,2]
x_sol_exp = traj[sim_data.num_q+sim_data.num_v+2:end,2]

# qnext = q0
# vnext = v0

τ_ip, x_sol_ip, λ_ip, μ_ip, fx_ip = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=true);

τ_auglag, x_sol_auglag, λ_auglag, μ_auglag, fx_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=false);

# display("Explicit")
# display(x_sol_exp)
display("IP")
display(x_sol_ip)
display("Aug Lag")
display(x_sol_auglag)

display("torques")
display(τ_ip)
display(τ_auglag)

elapsed time: 0.107861896 seconds
elapsed time: 0.02501572 seconds
elapsed time: 0.021753322 seconds
elapsed time: 

0.022307834 seconds
elapsed time: 0.033329561 seconds
elapsed time: 0.021151022 seconds
elapsed time: 0.021128369 seconds
elapsed time: 0.01971584 seconds
elapsed time: 0.023040776 seconds
elapsed time: 0.024520701 seconds
elapsed time: 0.023501186 seconds
elapsed time: 0.020516245 seconds
elapsed time: 0.025433699 seconds
elapsed time: 0.023808751 seconds
elapsed time: 0.023342625 seconds
elapsed time: 0.034493784 seconds
elapsed time: 0.021402504 seconds
Solve_Succeeded
Solve_Succeeded

"IP"

6-element Array{Float64,1}:
 0.0     
 0.0     
 0.500003
 0.0     
 0.475475
 1.00001 

"Aug Lag"

6-element Array{Float64,1}:
 1.08301e-8
 0.137487  
 0.362376  
 0.137487  
 0.00693156
 0.999971  

"torques"

6-element Array{Float64,1}:
  0.0        
  0.0        
  0.0        
  4.90503    
 -6.00693e-16
 -9.81006    

6-element Array{Float64,1}:
  0.0        
  0.0        
  0.0        
  3.55491    
 -5.44564e-16
 -9.80972    

In [43]:
τ_auglag, x_sol_auglag, λ_auglag, μ_auglag, fx_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=false);

# autodiff 
J_auto = ForwardDiff.jacobian(qv -> solve_implicit_contact_τ(sim_data,q0,v0,u0,qv[1:sim_data.num_q],qv[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)[2], vcat(qnext,vnext))

# numerical
ϵ = 1e-12
J_num = zeros(size(J_auto))
qv = vcat(qnext,vnext)
for i = 1:length(qv)
    δ = zeros(length(qv))
    δ[i] = ϵ
    J_num[:,i] = (solve_implicit_contact_τ(sim_data,q0,v0,u0,(qv+δ)[1:sim_data.num_q],(qv+δ)[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)[2] - x_sol_auglag)/ϵ
#     J_num[:,i] = solve_implicit_contact_τ(sim_data,q0,v0,u0,(qv+δ)[1:sim_data.num_q],(qv+δ)[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)[2]
end

display(J_auto)
display(J_num)

6×13 Array{Float64,2}:
 0.0  -1.41698e-18   1.19869e-7   3.95908e-12  …    6.03409e-12   2.4437e-6 
 0.0  -6.34783e-6    0.137864    -0.500001         10.1937        2.81067   
 0.0   2.56816e-12   0.112132     1.64156e-6       -1.22324e-6    2.28617   
 0.0   6.34783e-6    0.137864     0.499999        -10.1937        2.81067   
 0.0   3.70199e-12  -7.81412e-8   1.35824e-5        9.12105e-6   -4.34685e-6
 0.0  -2.00124e-12   1.0          3.65392e-6   …    3.19408e-6   20.3874    

6×13 Array{Float64,2}:
 0.0  -111.922      -75.2201     234.579      …  -295.379      -47.4133   
 0.0     2.87385e5   -1.73755e5    1.85326e6        2.87715e5    3.16456e5
 0.0     1.06236e8    7.05815e7   -2.20477e8        2.80897e8    4.398e7  
 0.0     2.87385e5   -1.73755e5    1.85326e6        2.87695e5    3.16456e5
 0.0    -1.69347e5    1.51338e6   -6.78617e6       -2.01147e6   -4.88994e5
 0.0     5.8256e6    -1.89592e6    6.60826e6  …     4.66386e6    1.95325e6

In [23]:
 (J_num[:,1] - x_sol_auglag)/ϵ

6-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [35]:
maximum(abs.(J_auto-J_num))

0.00011239110444094536

In [12]:
abs.(J_auto-J_num)

6×13 Array{Float64,2}:
 0.0  1.17929e8    5.54961e7    5.90191e7    0.0  …  4.85598e7   0.115786   
 0.0  1.17929e8    5.54961e7    5.90191e7    0.0     4.85598e7   0.115791   
 0.0  1.17929e8    5.54961e7    5.90191e7    0.0     4.85598e7   0.115786   
 0.0  1.17929e8    5.54961e7    5.90191e7    0.0     4.85598e7   0.115777   
 0.0  5.52014e-42  4.60942e-39  9.22638e-47  0.0     2.60672e-6  1.73472e-6 
 0.0  9.16002e-25  8.60679e-24  5.07938e-39  0.0  …  4.19766e-5  0.000109654

In [27]:
J_num

6×13 Array{Float64,2}:
 0.0  -5.53932e7   2.60583e7   1.46989e7  0.0  0.0  …   1.35127e7     2.76304
 0.0   5.54017e7  -2.60562e7  -1.46982e7  0.0  0.0     -1.35121e7     1.19871
 0.0  -5.53932e7   2.60583e7   1.46989e7  0.0  0.0      1.35127e7     2.76307
 0.0   5.54017e7  -2.60562e7  -1.46982e7  0.0  0.0     -1.35121e7     1.19871
 0.0   0.0         0.0         0.0        0.0  0.0      0.0           0.0    
 0.0   0.0         0.0         0.0        0.0  0.0  …  -0.000444089  20.3861 

In [28]:
J_auto

6×13 Array{Float64,2}:
 0.0  -3.30962e-15  -5.87669e-15   3.27191e-6   …   2.34824e-5   1.98036    
 0.0   3.3259e-15    5.91148e-15  -3.27238e-6       7.92411      1.98142    
 0.0  -3.30962e-15  -5.94641e-15   3.27191e-6       2.34824e-5   1.98036    
 0.0   3.29125e-15   5.91148e-15  -3.27238e-6      -7.92412      1.98142    
 0.0  -8.98363e-32  -1.21529e-31  -1.78426e-38      2.28571e-6   3.38714e-21
 0.0  -7.82746e-18  -4.70622e-18  -5.10436e-32  …   8.23815e-5  20.3864     